In [1]:
import scanpy as sc
import numpy as np
from tqdm import tqdm

In [2]:
def get_rank_score(
    pred_stim,
    true_ctrl,
):
    rank_scores = []
    for i_pred, pred in tqdm(enumerate(pred_stim)):
        all_euclideans = []
        for i_true, true in enumerate(true_ctrl):
                            
            euclidean = np.sqrt(((pred - true)**2).sum())
            all_euclideans.append(euclidean)

        ranks = np.argsort(all_euclideans)
        ranks = np.argsort(ranks)
        query_rank = ranks[i_pred] / (len(all_euclideans) - 1)

        rank_score = 1 - query_rank
        rank_scores.append(rank_score)

    return np.mean(rank_scores)

In [3]:
pred_path = '/data/Experiments/Benchmark/SCDISENTANGLE_REPRODUCE/Benchmarks/SCDISENTANGLE/Norman/predictions/combinatorially_seen/CBL+CNN1.h5ad'
adata_path = '/data/Experiments/Benchmark/SCDISENTANGLE_REPRODUCE/Datasets/preprocessed_datasets/norman.h5ad'

In [4]:
adata = sc.read_h5ad(adata_path)
adata_pred = sc.read_h5ad(pred_path)

sc.pp.normalize_total(adata, target_sum=adata.uns['single_perts_median'])
sc.pp.log1p(adata)

sc.pp.normalize_total(adata_pred, target_sum=adata.uns['single_perts_median'])
sc.pp.log1p(adata_pred)

/home/SE/miniconda3/envs/trials/lib/python3.10/site-packages/anndata/_core/anndata.py:1818: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/home/SE/miniconda3/envs/trials/lib/python3.10/site-packages/numba/np/ufunc/parallel.py:371: NumbaWarning: The TBB threading layer requires TBB version 2021 update 6 or later i.e., TBB_INTERFACE_VERSION >= 12060. Found TBB_INTERFACE_VERSION = 12050. The TBB threading layer is disabled.
  warnings.warn(problem)
adata.X seems to be already log-transformed.


In [5]:
true_ctrl = adata[adata.obs['cond_harm'] == 'ctrl'].X.toarray().copy()
pred_stim = adata_pred[adata_pred.obs['cond_harm'] == 'CBL+CNN1'].X.toarray().copy()

In [6]:
get_rank_score(
    pred_stim,
    true_ctrl,
)

11855it [11:34, 17.07it/s]


0.7477012637305123

In [7]:
adata_pred[adata_pred.obs['cond_harm'] == 'CBL+CNN1'].obs['sc_cell_ids']

AGACGTTGTCTAGCGC        26
AACGTTGAGCGTAGTG        45
TCCCGATGTAAACGCG        47
ACGCCGAGTGCGATAG        51
GCTGCAGGTAGATTAG        61
                     ...  
TGCGCAGGTCGAGATG    108522
TGCGTGGCAGCTGTTA    108525
TTCGAAGAGGTGATTA    108576
TTGAACGAGTCGAGTG    108589
TTTCCTCGTACGCACC    108611
Name: sc_cell_ids, Length: 11855, dtype: int64

In [8]:
all(adata[adata.obs['cond_harm'] == 'ctrl'].obs['sc_cell_ids'] == adata_pred[adata_pred.obs['cond_harm'] == 'CBL+CNN1'].obs['sc_cell_ids'])

True

In [9]:
all(adata_pred.obs['cond_harm_pred'] == adata_pred.obs['cond_harm'])

True

In [12]:
def to_np(arrs):
    """
    Convert a list/tuple of AnnData-like objects to a tuple of dense NumPy
    arrays taken from their `.X` attribute.
    """
    arrays = []
    for arr in arrs:
        x = arr.X
        if not isinstance(x, np.ndarray):
            x = x.toarray()
        arrays.append(x)
    return tuple(arrays)

In [15]:
def get_sc_similarity(
    _pred_stim, 
    _true_ctrl, 
    ):
   
    #_pred_stim, _true_ctrl = to_np([_pred_stim, _true_ctrl])
    
    assert _pred_stim.shape == _true_ctrl.shape, "Shape mismatch"
    N = _pred_stim.shape[0]
    assert N >= 2, ""

    # Pairwise euclidean (NxN)
    a2 = (_pred_stim**2).sum(axis=1, keepdims=True)          # (N,1)
    b2 = (_true_ctrl**2).sum(axis=1, keepdims=True).T        # (1,N)
    D2 = a2 + b2 - 2 * (_pred_stim @ _true_ctrl.T)
    D2 = np.maximum(D2, 0.0)
    D = np.sqrt(D2)

    # Single-cell preservation metric
    ranks = np.argsort(D, axis=1).argsort(axis=1)[np.arange(N), np.arange(N)]
    rank_scores = 1.0 - ranks / (N - 1)         # in [0,1]
    rank_mean = float(rank_scores.mean())

    metrics = {
        'All': {
            'Single-cell Identity preservation': rank_mean*100,
        }
    }
    return metrics

In [16]:
get_sc_similarity(
    pred_stim,
    true_ctrl
)

{'All': {'Single-cell Identity preservation': 74.77012281507108}}

In [1]:
import scanpy as sc

In [2]:
adata = sc.read_h5ad(
    '/data/Experiments/Benchmark/SCDISENTANGLE_REPRODUCE/Datasets/preprocessed_datasets/norman.h5ad'
)

In [3]:
adata_mean = sc.get.aggregate(
    adata,
    by="cond_harm",
    func="mean"
)

In [4]:
adata_mean

AnnData object with n_obs × n_vars = 230 × 5446
    obs: 'cond_harm'
    var: 'ensemble_id', 'ncounts', 'ncells', 'n_counts', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    layers: 'mean'

In [14]:
adata_mean.layers['mean'][0].mean()
indices = adata_mean.obs_names.tolist()

In [17]:
indices.index('CBL+CNN1')

16

In [12]:
adata[adata.obs['cond_harm'] == 'AHR'].X.mean(axis=0).mean()

1.5582889